In [1]:
import sys
sys.path.insert(0, "../")

from ctgan.synthesizers.ctgan import CTGANSynthesizer
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier

from src.data_loader import load_seer_cutract_dataset
from src.utils import *


/home/nabeel/miniconda3/envs/3s_final_env/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Get Data

In [2]:
import numpy as np
import pandas as pd
from sklearn.utils import shuffle

seed = 0
df = pd.read_csv("../data/Base.csv")
for col in [
    "payment_type",
    "employment_status",
    "housing_status",
    "source",
    "device_os",
]:
    df[col] = df[col].astype("category").cat.codes

df.rename(columns={"fraud_bool": "y"}, inplace=True)

mask = df["y"] == True
df_fraud = df[mask]
df_no = df[~mask]

n_samples = 10000
df = pd.concat(
    [
        df_fraud.sample(n_samples, random_state=seed),
        df_no.sample(n_samples, random_state=seed),
    ]
)

df = shuffle(df, random_state=seed)

X_train = df[df["month"] < 6]
X_test = df[df["month"] >= 6]


# Train base models

In [3]:
from copy import deepcopy

from sklearn.metrics import accuracy_score, roc_auc_score

model_dict = {
    "mlp": MLPClassifier(),
    "knn": KNeighborsClassifier(),
    "dt": DecisionTreeClassifier(),
    "rf": RandomForestClassifier(),
    "gbc": GradientBoostingClassifier(),
}

trained_model_dict = {}

for model in model_dict.keys():
    clf = model_dict[model]
    clf.fit(X_train.drop("y", axis=1), X_train["y"])

    trained_model_dict[model] = deepcopy(clf)


# Train Generative model

In [4]:
discrete_columns = [
    "y",
    "payment_type",
    "employment_status",
    "housing_status",
    "source",
    "device_os",
]


syn_model = CTGANSynthesizer(
    embedding_dim=128,
    generator_dim=(256, 256),
    discriminator_dim=(256, 256),
    generator_lr=2e-4,
    generator_decay=1e-6,
    discriminator_lr=2e-4,
    discriminator_decay=1e-6,
    batch_size=500,
    discriminator_steps=1,
    log_frequency=True,
    verbose=False,
    epochs=300,
    pac=10,
    cuda=True,
)

seed_everything(seed)
syn_model.set_random_state(seed)
syn_model.fit(train_data=X_train, discrete_columns=discrete_columns)


# Identify column of the marginal to shift

In [5]:
from numpy import ma
from tqdm import tqdm

metric = "session_length_in_minutes"
data = X_train[metric]
cat_groups_present = False

if len(np.unique(data)) < 10:
    cat_groups = np.unique(data)
    cat_groups_present = True
else:
    mean, std = np.mean(data), np.std(data)

    minimum, maximum = np.min(data), np.max(data)

eval_idx = np.where(X_train.columns == metric)[0][0]
eval_idx


26

# Shift 3S

In [6]:
from src.shift import rejection_sample

ys_mlp_all = []
ys_knn_all = []
ys_dt_all = []
ys_rf_all = []
ys_gbc_all = []

cut_off = X_train[metric].quantile([0.25]).values[0]

for i in range(2):

    ys_mlp_tmp = []
    ys_knn_tmp = []
    ys_dt_tmp = []
    ys_rf_tmp = []
    ys_gbc_tmp = []
    n_range = 10
    n_std = 1 * std

    shift_df, _ = syn_model.sample(n=10000, shift=False)

    xs = list(
        np.arange(
            mean - n_std, mean + n_std, ((mean + n_std) - (mean - n_std)) / n_range
        )
    )  #
    for shift_mean in np.arange(
        mean - n_std, mean + n_std, ((mean + n_std) - (mean - n_std)) / n_range
    ):  #

        if shift_mean < cut_off:
            continue

        reject_df = rejection_sample(
            D=shift_df, mean=shift_mean, std=std / 2, feat_id=[eval_idx]
        )
        if len(reject_df) == 0:
            continue
        test_df = pd.DataFrame(reject_df, columns=X_test.columns)
        real_tester = test_df
        for model in model_dict.keys():
            clf = model_dict[model]
            y_score = clf.predict_proba(real_tester.drop("y", axis=1))[:, 1]
            y_pred = clf.predict(real_tester.drop("y", axis=1))

            if model == "mlp":
                ys_mlp_tmp.append(accuracy_score(real_tester["y"], y_pred))

            if model == "knn":
                ys_knn_tmp.append(accuracy_score(real_tester["y"], y_pred))

            if model == "dt":
                ys_dt_tmp.append(accuracy_score(real_tester["y"], y_pred))

            if model == "rf":
                ys_rf_tmp.append(accuracy_score(real_tester["y"], y_pred))

            if model == "gbc":
                ys_gbc_tmp.append(accuracy_score(real_tester["y"], y_pred))

    ys_mlp_all.append(ys_mlp_tmp)
    ys_knn_all.append(ys_knn_tmp)
    ys_dt_all.append(ys_dt_tmp)
    ys_rf_all.append(ys_rf_tmp)
    ys_gbc_all.append(ys_gbc_tmp)

xs = np.array(xs)[xs > cut_off]


# Rejection sample (Test/Oracle data)

In [7]:
yr_mlp = []
yr_knn = []
yr_dt = []
yr_rf = []
yr_gbc = []
xr = list(
    np.arange(mean - n_std, mean + n_std, ((mean + n_std) - (mean - n_std)) / n_range)
)
i = 0
for shift_mean in np.arange(
    mean - n_std, mean + n_std, ((mean + n_std) - (mean - n_std)) / n_range
):

    if shift_mean < cut_off:
        continue
    reject_df = rejection_sample(
        D=X_test, mean=shift_mean, std=std / 2, feat_id=[eval_idx]
    )
    if len(reject_df) == 0:
        continue
    test_df = pd.DataFrame(reject_df, columns=X_test.columns)
    real_tester = test_df
    for model in model_dict.keys():
        clf = model_dict[model]
        y_score = clf.predict_proba(real_tester.drop("y", axis=1))[:, 1]
        y_pred = clf.predict(real_tester.drop("y", axis=1))

        if model == "mlp":
            yr_mlp.append(accuracy_score(real_tester["y"], y_pred))

        if model == "knn":
            yr_knn.append(accuracy_score(real_tester["y"], y_pred))

        if model == "dt":
            yr_dt.append(accuracy_score(real_tester["y"], y_pred))

        if model == "rf":
            yr_rf.append(accuracy_score(real_tester["y"], y_pred))

        if model == "gbc":
            yr_gbc.append(accuracy_score(real_tester["y"], y_pred))


xr = np.array(xr)[xr > cut_off]


# Shift RS (Source)

In [8]:
yr_mlp_val = []
yr_knn_val = []
yr_dt_val = []
yr_rf_val = []
yr_gbc_val = []
xr = list(
    np.arange(mean - n_std, mean + n_std, ((mean + n_std) - (mean - n_std)) / n_range)
)
i = 0
for shift_mean in np.arange(
    mean - n_std, mean + n_std, ((mean + n_std) - (mean - n_std)) / n_range
):

    if shift_mean < cut_off:
        continue
    reject_df = rejection_sample(
        D=X_train, mean=shift_mean, std=std / 2, feat_id=[eval_idx]
    )
    if len(reject_df) == 0:
        continue
    test_df = pd.DataFrame(reject_df, columns=X_train.columns)
    real_tester = test_df
    for model in model_dict.keys():
        clf = model_dict[model]
        y_score = clf.predict_proba(real_tester.drop("y", axis=1))[:, 1]
        y_pred = clf.predict(real_tester.drop("y", axis=1))

        if model == "mlp":
            yr_mlp_val.append(accuracy_score(real_tester["y"], y_pred))

        if model == "knn":
            yr_knn_val.append(accuracy_score(real_tester["y"], y_pred))

        if model == "dt":
            yr_dt_val.append(accuracy_score(real_tester["y"], y_pred))

        if model == "rf":
            yr_rf_val.append(accuracy_score(real_tester["y"], y_pred))

        if model == "gbc":
            yr_gbc_val.append(accuracy_score(real_tester["y"], y_pred))


xr = np.array(xs)[xs > cut_off]


# Mean Shift

In [9]:
yr_mlp_ms = []
yr_knn_ms = []
yr_dt_ms = []
yr_rf_ms = []
yr_gbc_ms = []
xr = list(
    np.arange(mean - n_std, mean + n_std, ((mean + n_std) - (mean - n_std)) / n_range)
)
i = 0
for shift_mean in np.arange(
    mean - n_std, mean + n_std, ((mean + n_std) - (mean - n_std)) / n_range
):
    from copy import deepcopy

    if shift_mean < cut_off:
        continue
    test_df = deepcopy(X_train)
    test_df[metric] = np.random.normal(
        loc=shift_mean, scale=std / 2, size=len(X_train[metric])
    )
    if len(reject_df) == 0:
        continue

    real_tester = test_df
    for model in model_dict.keys():
        clf = model_dict[model]
        y_score = clf.predict_proba(real_tester.drop("y", axis=1))[:, 1]
        y_pred = clf.predict(real_tester.drop("y", axis=1))

        if model == "mlp":
            yr_mlp_ms.append(accuracy_score(real_tester["y"], y_pred))

        if model == "knn":
            yr_knn_ms.append(accuracy_score(real_tester["y"], y_pred))

        if model == "dt":
            yr_dt_ms.append(accuracy_score(real_tester["y"], y_pred))

        if model == "rf":
            yr_rf_ms.append(accuracy_score(real_tester["y"], y_pred))

        if model == "gbc":
            yr_gbc_ms.append(accuracy_score(real_tester["y"], y_pred))


xs = np.array(xs)[xs > cut_off]


# Compare to performance on oracle/test

In [10]:
ids = np.where((X_train[metric] > xs[0]) & (X_train[metric] < xs[-1]))
quantiles = X_train[metric].iloc[ids].quantile([0.25, 0.5, 0.75]).values
q1 = xs < quantiles[0]
q2 = (xs > quantiles[0]) & (xs < quantiles[2])
q3 = xs > quantiles[2]


results = {}

q1_dict = {}
q1_dict["Error 3S"] = np.mean(np.abs(np.mean(ys_rf_all, axis=0) - yr_rf)[q1])
q1_dict["Error MS"] = np.mean(np.abs(np.array(yr_rf_ms) - np.array(yr_rf))[q1])
q1_dict["Error RS"] = np.mean(np.abs(np.array(yr_rf_val) - np.array(yr_rf))[q1])

q2_dict = {}
q2_dict["Error 3S"] = np.mean(np.abs(np.mean(ys_rf_all, axis=0) - yr_rf)[q2])
q2_dict["Error MS"] = np.mean(np.abs(np.array(yr_rf_ms) - np.array(yr_rf))[q2])
q2_dict["Error RS"] = np.mean(np.abs(np.array(yr_rf_val) - np.array(yr_rf))[q2])

q3_dict = {}
q3_dict["Error 3S"] = np.mean(np.abs(np.mean(ys_rf_all, axis=0) - yr_rf)[q3])
q3_dict["Error MS"] = np.mean(np.abs(np.array(yr_rf_ms) - np.array(yr_rf))[q3])
q3_dict["Error RS"] = np.mean(np.abs(np.array(yr_rf_val) - np.array(yr_rf))[q3])

results["Q1"] = q1_dict
results["Q2"] = q2_dict
results["Q3"] = q3_dict


threeS_err = np.abs(np.mean(ys_rf_all, axis=0) - yr_rf)
avg_dict = {}
avg_dict["Error 3S"] = np.mean(threeS_err)
avg_dict["Error MS"] = np.mean(np.abs(np.array(yr_rf_ms) - np.array(yr_rf)))
avg_dict["Error RS"] = np.mean(np.abs(np.array(yr_rf_val) - np.array(yr_rf)))

results["avg"] = avg_dict


In [11]:
ids = np.where((X_train[metric] > xs[0]) & (X_train[metric] < xs[-1]))
quantiles = X_train[metric].iloc[ids].quantile([0.25, 0.5, 0.75]).values
q1 = xs < quantiles[0]
q2 = (xs > quantiles[0]) & (xs < quantiles[2])
q3 = xs > quantiles[2]


results = {}

q1_dict = {}
q1_dict["Error 3S"] = np.mean(np.abs(np.mean(ys_rf_all, axis=0) - yr_rf)[q1])
q1_dict["Error MS"] = np.mean(np.abs(np.array(yr_rf_ms) - np.array(yr_rf))[q1])
q1_dict["Error RS"] = np.mean(np.abs(np.array(yr_rf_val) - np.array(yr_rf))[q1])

q2_dict = {}
q2_dict["Error 3S"] = np.mean(np.abs(np.mean(ys_rf_all, axis=0) - yr_rf)[q2])
q2_dict["Error MS"] = np.mean(np.abs(np.array(yr_rf_ms) - np.array(yr_rf))[q2])
q2_dict["Error RS"] = np.mean(np.abs(np.array(yr_rf_val) - np.array(yr_rf))[q2])

q3_dict = {}
q3_dict["Error 3S"] = np.mean(np.abs(np.mean(ys_rf_all, axis=0) - yr_rf)[q3])
q3_dict["Error MS"] = np.mean(np.abs(np.array(yr_rf_ms) - np.array(yr_rf))[q3])
q3_dict["Error RS"] = np.mean(np.abs(np.array(yr_rf_val) - np.array(yr_rf))[q3])

results["Q1"] = q1_dict
results["Q2"] = q2_dict
results["Q3"] = q3_dict


threeS_err = np.abs(np.mean(ys_rf_all, axis=0) - yr_rf)
avg_dict = {}
avg_dict["Error 3S"] = np.mean(threeS_err)
avg_dict["Error MS"] = np.mean(np.abs(np.array(yr_rf_ms) - np.array(yr_rf)))
avg_dict["Error RS"] = np.mean(np.abs(np.array(yr_rf_val) - np.array(yr_rf)))

results["avg"] = avg_dict

results


{'Q1': {'Error 3S': 0.03650000000000009,
  'Error MS': 0.1861331769611675,
  'Error RS': 0.18899999999999995},
 'Q2': {'Error 3S': 0.03600000000000003,
  'Error MS': 0.19916575449569973,
  'Error RS': 0.20199999999999996},
 'Q3': {'Error 3S': 0.06787500000000005,
  'Error MS': 0.1793619364086526,
  'Error RS': 0.18275},
 'avg': {'Error 3S': 0.05428571428571433,
  'Error MS': 0.18598749022673963,
  'Error RS': 0.1891428571428571}}